# Introduction to DataFrames.jl

Loosely based on [this
tutorial](https://juliaai.github.io/DataScienceTutorials.jl/data/dataframe/)
from Data Science Tutorials.

DataFrames.jl **cheatsheets**:
[english](https://www.ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1.pdf),
[chinese](https://www.ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1_zh.pdf)

## Setup

Inspect Julia version:

In [1]:
VERSION

v"1.6.5"

The following instantiates a package environment.

The package environment has been created using **Julia 1.6** and may not
instantiate properly for other Julia versions.

In [2]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating environment at `~/GoogleDrive/Julia/HelloJulia/notebooks/02_dataframes/Project.toml`
Precompiling project...
  ✓ SpecialFunctions
  ✓ StatsFuns
  ✓ Distributions
  ✓ ScientificTypes
  ✓ OpenML
  5 dependencies successfully precompiled in 19 seconds (52 already precompiled)


## Loading some demonstration data

We'll be using [OpenML](https://www.openml.org/home) to grab datasets.

In [3]:
using OpenML
using DataFrames

table = OpenML.load(42638); # Titanic data set
typeof(table)

┌ Info: Precompiling OpenML [8b6db2d4-7670-4922-a472-f9537c81ab66]
└ @ Base loading.jl:1342


Tables.DictColumnTable

This is not a `DataFrame`. However, like
[many](https://github.com/JuliaData/Tables.jl/blob/main/INTEGRATIONS.md)
other tabular data containers in Julia, it can be converted to one in the
obvious way:

In [4]:
df = DataFrame(table);

A `DataFrame` is essentially just a wrapper around a number of
vectors with names, conceptualized as a table with the vectors as
columns.

Lets' look the first few rows (observations) of `df`:

In [5]:
first(df, 4)

,pclass,sex,age,sibsp,fare,cabin,embarked,survived
,Cat…,Cat…,Float64,Float64,Float64,Cat…?,Cat…?,Cat…
1,3,male,22.0,1.0,7.25,missing,S,0
2,1,female,38.0,1.0,71.2833,C85,C,1
3,3,female,26.0,0.0,7.925,missing,S,1
4,1,female,35.0,1.0,53.1,C123,S,1


Here's a summary of stats for each column:

In [6]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,pclass,,1,,3,0,"CategoricalValue{String, UInt32}"
2,sex,,female,,male,0,"CategoricalValue{String, UInt32}"
3,age,29.7589,0.42,30.0,80.0,0,Float64
4,sibsp,0.523008,0.0,0.0,8.0,0,Float64
5,fare,32.2042,0.0,14.4542,512.329,0,Float64
6,cabin,,E31,,C148,687,"Union{Missing, CategoricalValue{String, UInt32}}"
7,embarked,,C,,S,2,"Union{Missing, CategoricalValue{String, UInt32}}"
8,survived,,0,,1,0,"CategoricalValue{String, UInt32}"


To get just the column names, use `names(df)`.

## Selecting a single entry:

In [7]:
number = df[5, :age]   # or df[5, 4]

35.0

## Selecting a single row:

We use the wild-card `:`

In [ ]:
row = df[5, :]

In [ ]:
number = row.age

## Selecting multiple rows

By index:

In [ ]:
small_df = df[3:7, :]

By applying some criterion:

In [ ]:
df2 = filter(df) do row
    row.age > 60 && row.survived == "1"
end

(Alternatively, one can use `subset` or Boolean indexing; see the
[cheatsheet](https://www.ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1.pdf).)

## Selecting a single column

Here's how I get the column named "Age":

In [ ]:
df2.age

Since this is just a vector, I can restrict the rows as usual:

In [ ]:
df2.age[2:5]  # but same as df2[2:5, :Age]

## Selecting multiple columns - `select`

The following wraps `df.age` as a single-column `DataFrame`:

In [ ]:
select(df2, :age)

... which generalizes to multiple columns:

In [ ]:
select(df2, [:age, :sex])

In [ ]:
select(df2, Not(:age))

In [ ]:
select(df2, Between(:sex, :fare))

## Copies or views of the data?

So far, all the selection mechanisms discussed so far return a new
`DataFrame` object, with these exceptions:

* selecting a single row, as in `df[3,:]` - this returns a `DataFrameRow`.
* selecting a single column as a vector, as in `df.age` - this returns a vector

These exceptions are *views* of the original `DataFrame`, in the
sense that no data is copied and mutating the view mutates the
original `DataFrame`:

In [ ]:
v = df.age;
v[1] = 1000
df[1, :age]

Mutating the output of all the other selection mechanisms will not
touch the original `DataFrame`, as data is copied in those cases.

Use `select!` to make "in-place" column selections which mutate the
original `DataFrame`. For example, the following permanently drops a
column from `df`:

In [ ]:
select!(df, Not(:cabin));
first(df, 2)

In addition to row and column views of a `DataFrame` we can construct
larger views, of type `SubDataFrame`, which share most of the
behviour of an ordinary `DataFrame`, except they only *point* to
data in the parent `DataFrame`, rather than duplicate it:

In [ ]:
df_copy = df[1:3, [:fare, :age]]       # this is copy
df_view = @view df[1:3, [:fare, :age]] # this is a view

In [ ]:
df_view[1, :age] = 4000
df[1, :age]

In [ ]:
df_copy[1, :age]

## Describing the data

To broaden the summary provided by `describe(df)` we can pass a
number of symbols to indicate the statistics we are after:

In [ ]:
describe(df, :min, :max, :mean, :median, :std)

The following are all supported:
* `mean`, `std`, `min`, `max`, `median`, `first`, `last` are all fairly self explanatory
* `q25`, `q75` are respectively for the 25th and 75th percentile,
* `eltype`, `nunique`, `nmissing` can also be used

You can also pass your custom function with a pair `function =>
:name` for instance:

In [ ]:
using Statistics # to get functions like `mean` and `std`
foo(v) = mean(abs.(v))
d = describe(df, :mean, :median, foo => :mae)
first(d, 3)

Note that the object returned by `describe` is itself a `DataFrame`:

In [ ]:
select(d, [:variable, :mean])

## Materializing as a matrix

To convert the content of the dataframe as one big matrix do this:
use `convert`:

In [ ]:
mat = Matrix(df)
mat[1:3, 1:3]

## Adding columns

Adding a column to a dataframe is very easy:

In [ ]:
df.weird = df.fare ./ df.age;
first(df, 3)

That's it! Remember also that you can drop columns or make
subselections with `select` and `select!`.

The remainder of this tutorial is an abbreviated version of a
[DataFrames.jl
tutorial](http://juliadata.github.io/DataFrames.jl/latest/man/split_apply_combine/).

In [ ]:
iris = OpenML.load(61) |> DataFrame;
first(iris, 3)

## `groupby`

The `groupby` function allows to form "sub-dataframes" corresponding
to groups of rows.  This can be very convenient to run specific
analyses for specific groups without copying the data.

The basic usage is `groupby(df, cols)` where `cols` specifies one or
several columns to use for the grouping.

Consider a simple example: in `iris` there is a `Species` column
with 3 species:

In [ ]:
unique(iris.class)

We can form views for each of these:

In [ ]:
gdf = groupby(iris, :class);

The `gdf` object now corresponds to **views** of the original
dataframe for each of the 3 species; the first species
is `"Iris-setosa"` with:

In [ ]:
subdf_setosa = gdf[1];
typeof(subdf_setosa)

In [ ]:
describe(subdf_setosa, :min, :mean, :max)

Recall that this means modifying `subdf_setosa` also modifies its
parent `iris`.

Do `?groupby` for more information.

## `combine`

The `combine` function allows to derive a new dataframe out of
transformations of an existing one.  Here's an example taken from
the official doc (see `?combine`):

In [ ]:
df = DataFrame(a=1:3, b=4:6)

In [ ]:
combine(df, :a => sum, nrow)

What happened here is that the derived DataFrame has two columns
obtained respectively by (1) computing the sum of the first column
and (2) applying the `nrow` function on the `df`.

The transformation can produce one or several values, and `combine` will
try to accomodate this with appropriate packing:

In [ ]:
foo(v) = v[1:2]
combine(df, :a => maximum, :b => foo)

Here the maximum value of `a` is copied twice so that the two
columns have the same number of rows.

In [ ]:
bar(v) = v[end-1:end]
combine(df, :a => foo, :b => bar)

## `combine` with `groupby`

Combining `groupby` with `combine` is very useful.  For instance you
might want to compute statistics across groups for different
variables:

In [ ]:
combine(groupby(iris, :class), :petallength => mean)

Let's break this operatioin down:

1. The `groupby(iris, :class)` creates groups using the `:class` column (which has values `Iris-setosa`, `Iris-versicolor`, `Iris-virginica`)
2. The `combine` creates a derived dataframe by applying the `mean` function to the `:petallength` column
3. Since there are three groups, we get one column (mean of `petallength`) and three rows (one per group).


You can do this for several columns/statistics at the time and give
new column names to the results:

In [ ]:
gdf = groupby(iris, :class)
combine(gdf, :petallength => mean => :MPL, :petallength => std => :SPL)

So here we assign the names `:MPL` and `:SPL` to the derived
columns.  If you want to apply something on all columns apart from
the grouping one, using `names` and `Not` comes in handy:

In [ ]:
combine(gdf, names(iris, Not(:class)) .=> std)

where

In [ ]:
names(iris, Not(:class))

and note the use of `.` in `.=>` to indicate that we broadcast the
function over each column.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*